In [1]:
import telepot
from telepot.loop import MessageLoop
from telepot.namedtuple import InlineKeyboardMarkup, InlineKeyboardButton, ReplyKeyboardMarkup, KeyboardButton
from telepot.delegate import (per_chat_id, per_application, call, create_open, pave_event_space)
from pprint import pprint
import sys
import time
import response_manager
import pandas as pd
%run Result_Word2Vec.ipynb
#import tfidf
#import scipy.sparse
#import joblib
#import response_manager

In [2]:
# Simulate a database to store unread messages
class MessageStore(object):
    def __init__(self):
        self._db = {}
        self._results_db = []
        self.tag = []
     

    def put_tag(self, t):
        if len(self.tag):
            #del self.tag
            self.tag.append(t)
        else:
            self.tag.append(t)
       
    def pull_tag(self):
        t = self.tag
        return t 
    
    def put_results(self, results):
        if len(self._results_db):
            del self._results_db[0]
        self._results_db.append(results)
    
    
            
    def pull_results(self):
        if len(self._results_db):
            res = self._results_db
            return res
        else:
            return []

    def put(self, msg):
        chat_id = msg['chat']['id']

        if chat_id not in self._db:
            self._db[chat_id] = []

        self._db[chat_id].append(msg)

    # Pull all unread messages of a `chat_id`
    def pull(self, chat_id):
        
        if len(self._db):
            messages = self._db[chat_id]
            # sort by date
            messages.sort(key=lambda m: m['date'])
            return messages
        else:
            return []
            

    
    def delete_mess(self, chat_id):
        if len(self._db):
            del self._db[chat_id]
        # Tells how many unread messages per chat_id
        #return [(k,len(v)) for k,v in self._db.items()]

In [3]:
# Accept commands from owner. Give him unread messages.
class MessageHandler(telepot.helper.ChatHandler):
    def __init__(self, seed_tuple, store, **kwargs):
        super(MessageHandler, self).__init__(seed_tuple, **kwargs)
        self._store = store

    def _read_messages(self, messages):
        for msg in messages:
            # assume all messages are text
            self.sender.sendMessage(msg['text'])

    def on_chat_message(self, msg):
        
        #Flag per non entrare nella iterazione del yes or no quando si sta processando per la prima volta il problema 
        flag = True
        
        print(msg)
        content_type, chat_type, chat_id = telepot.glance(msg)
        
        if content_type == 'text' :
            name = msg["from"]["first_name"]
            txt = str(msg['text'])
        
        else:
            self.sender.sendMessage("I don't understand")
            return
        print (txt)
        
        #Con questo metodo creiamo una lista con i messaggi precedenti relativi alla chat con l'utente corrente 
        message_history = self._store.pull(chat_id)
          
        #Se il comando è start mando il messaggio di inzio ed elimino i messaggi precedenti salvati relativi all'utente 
        if '/start' == txt :
            #print(message_history)
            self.sender.sendMessage('Hello {}! I\'m StackBot. Tell me what is your problem involving Python language, try to be as general as possible'.format(name))
            self._store.delete_mess(chat_id)
            self._store.put(msg)
            
            if len(message_history) > 0 and txt == '/start':
                self._store.delete_mess(chat_id)
                message_history = self._store.pull(chat_id) 
                self._store.put(msg)
            
            
        elif len(message_history) == 0 and txt != '/start':
            
            markup = ReplyKeyboardMarkup(keyboard=[['Time', KeyboardButton(text='Logo')],["File", "Audio"]])
            
            #markup = InlineKeyboardButton(inline_keyboard=[['Logo','Text']])
            
            
            self.sender.sendMessage('You have to write /start to begin the interaction', 
                                    reply_markup = markup)
            
            
        
        #print("Message History", message_history)
        #print("Len Message History", len(message_history))
        
        #or (message_history[len(message_history)]['text'] and message_history[0]['text'] == '/start')
        
        #Se l'unico messaggio salvato è il comando start procediamo con il primo passo del processo
        if (len(message_history) == 1 and message_history[0]['text'] == '/start') :
            
            #controlliamo che il messaggio appena inviato non sia un altro comando 
            if 'entities' in msg:
                self._store.delete_mess(chat_id)
                self.sender.sendMessage("You can't type this in your request, please restart your interaction")
                
            #altrimenti sarà il testo del problema da mandare al w2v
            else:
                # resp = response_manager.gen_response(txt)
                #Chiamo la funzione che mi rende la lista dei tags delle domande con maggiore similarità
                
                results = w2v_result(txt, None)
                self._store.put_results(results)
                
                #print('RESULTS', results)
                
                #Salviamo nel nostro "db" i risultati del w2v e il messaggio relativo al problema dell'utente
               
                self._store.put(msg)
                
                #Chiedo all'utente se il suo problema ha a che fare con il tag appena estratto 
                #Notare che precedentemente sono stati eliminati i tag corrispondenti a parole già scritte dall'utente 
                #nella sua descrizione del problema
            
                sendControl(self, results, self._store, len(message_history), txt)
                
                flag = False
        
        #Analizzo il caso in cui sono presenti i messaggi relativi a /start e alla descrizione del problema
        #Quindi qui vado a scremare i risultati in base all'input dell'utente e al fatto se il tag a lui proposto 
        #centri o meno col suo problema 
        
        if len(message_history) > 1 and flag:
            res = []
            #Caso affermativo 
            if txt == 'Y' or txt == 'y':
                res = filter_results(self._store,1)
                self._store.put_results(res)
                sendControl(self, res, self._store, len(message_history), txt)
                
                #if(len(message_history))
                
            elif txt == 'N' or txt == 'n':
                res = filter_results(self._store,2)
                self._store.put_results(res)
                sendControl(self, res, self._store, len(message_history), txt)
                
            elif txt == 'C' or txt == 'c':
                res = self._store.pull_results()
                res[0].pop(0)
                self._store.put_results(res[0])
                sendControl(self, res[0], self._store, len(message_history), txt)
            
            elif txt == 'R' or txt == 'r':
                res = self._store.pull_results()
                res[0].pop(0)
                self._store.put_results(res[0])
                sendControl(self, res[0], self._store, len(message_history), txt)
                
                
                
            self._store.put(msg)
            
                

In [4]:
def sendControl(self, results, selfStore, message_history, txt):
    
    flag = False

    # Se per la richiesta non c'è nessun risultato
    if(message_history == 1 and len(results) == 0):
        self.sender.sendMessage("Error, no match with your request.\nClick /start and try again with another question!")
    
    
    if(message_history > 1):
        
        #Se ha rifiutato tutte le possibili soluzioni  
        if(len(results) == 0):
            self.sender.sendMessage("Error, there aren't other match with your requests.\nClick /start and try again with another question!")
        
        # Se results ha almeno una potenziale risposta    
        else:
            '''
            Viene proposta una risposta dopo due iterazioni con l'utente e successivamente 
            dopo ogni iterazione fino a quando non ci sono più risposte disponibili
            '''
            if(message_history%2 != 0):
                #print(results)
                self.sender.sendMessage("The solution to your problem is likely to have been discussed on this page: https://stackoverflow.com/questions/" + str(results[0][0]) + "/")
                self.sender.sendMessage("Click c if you want to continue the search")
            
            
            else:
                
                #Se c'è almeno una potenziale risposta
                if(len(results) != 0):
                    
                    #Se c'è solo una potenziale risposta
                    if(len(results) == 1):
                        self.sender.sendMessage("The solution to your problem is likely to have been discussed on this page: https://stackoverflow.com/questions/" + str(results[0][0]) + "/")
                        self.sender.sendMessage("If the answer is not the one you wanted click /start and and try again with another question!\n")
                        tags = []
                        flag = None
                    #Se vi è più di una potenziale risposta
                    else:
                        
                        #dai risultati prelevo tutte le liste dei tag delle varie domande 
                        tags = [el[2] for el in results]
                        #print(tags)
                        
                #Se non ci sono risposte 
                else: 
                    tags = []
   
                    
                #ciclo su ogni lista di tag, e mi fermo quando trovo il primo tag
                if len(tags) > 0:
                    for tag in tags:
                        #Se l'array contiene dei valori
                        if len(tag) > 0:
                            t = tag[0]
                            break
                        #Se l'array è vuoto    
                        else:
                            flag = True
                '''
                Se l'array precedentemente analizzato non è vuoto ma conteneva dei tag.
                Mostro all'utente un altro tag
                '''
                if(flag == False):
                    selfStore.put_tag(t)
                    
                    self.sender.sendMessage("Your question has something to do with:" + t + "?")
                    self.sender.sendMessage("Write Y for yes or N for no")
                    
                #Se l'array è vuoto     
                elif(flag == True):
                    #Se results ha solo una risposta
                    if(len(results) == 1):
                        self.sender.sendMessage("The solution to your problem is likely to have been discussed on this page: https://stackoverflow.com/questions/" + str(results[0][0]) + "/")
                        self.sender.sendMessage("If the answer is not the one you wanted click /start and and try again with another question!\n")
                    #Se results ha più di una risposta
                    else:
                        self.sender.sendMessage("The solution to your problem is likely to have been discussed on this page: https://stackoverflow.com/questions/" + str(results[0][0]) + "/")
                        self.sender.sendMessage("If the answer is not the one you wanted click R")
                        

In [5]:
def filter_results(store, choice):
        #Carico in variabili i precedenti risultati e il tag relativo
        tmp_results = store.pull_results()
        results = tmp_results[0]
        tmp_tag_target = store.pull_tag()
        tag_target= tmp_tag_target[len(tmp_tag_target) - 1] 
        
        #print(tmp_tag_target)
        tags = []
        
        #dai risultati prelevo tutte le liste dei tag delle varie domande

        tags = [el[2] for el in results]
            
        #print('TAGSssssss',tags)
        
        i = 0
        
        
        if len(tags) > 0:
            for tag in tags:
                #choice == 1 vuol dire che l'utente è interessato al tag
                if choice == 1: #Y
                    #Se il tag target non è presente nella lista dei tag relativi a questo risultato
                    if tag_target not in tag:
                        #print('\n')
                        #print('Removing the result', i)
                        #rimuovi la riga relativa dai risultati
                        
                        results.pop(i)
                        tmp_tag_target = tag_target
                        #i = i-1
                        #print('DOPO IL POP', results)
                    else:
                        i = i+1
                
                if choice == 2: #N
                    #Se il tag target è presente nella lista dei tag rimuovi la lista relativa a quel tag dai risultati 
                    if tag_target in tag:
                        #print('Removing the result', i)
                        results.pop(i)
                    else:
                        i = i+1
                                   
        #print('FINAlllL', results)
        
        j = 0
        
        if(choice == 1):
            for item in results:
                if tmp_tag_target in item[2]:
                    results[j][2].remove(tag_target)
                j = j+1    
        
        return results

In [6]:
class ChatBox(telepot.DelegatorBot):
    def __init__(self, token, owner_id):
        self._owner_id = owner_id
        self._seen = set()
        self._store = MessageStore()
        
        super(ChatBox, self).__init__(token, [
            pave_event_space()(
                per_chat_id(), create_open, MessageHandler, self._store, timeout=20),
        ])

In [ ]:
OWNER_ID = 102199992
TOKEN = '2067569208:AAHYKIRPZJ_eKtEFAtAmcFOWGlOMyRbGsXI'

bot = ChatBox(TOKEN, OWNER_ID)
MessageLoop(bot).run_as_thread()
print('Listening ...')

while 1:
    time.sleep(3)

Listening ...
{'message_id': 2086, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487658, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}
/start


ERROR:root:on_close() called due to IdleTerminate: 20


{'message_id': 2088, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487756, 'text': 'how create request to mongodb'}
how create request to mongodb
{'message_id': 2091, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487776, 'text': 'n'}
n
{'message_id': 2094, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487781, 'text': 

ERROR:root:on_close() called due to IdleTerminate: 20


{'message_id': 2097, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487806, 'text': 'c'}
c
{'message_id': 2100, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487818, 'text': 'r'}
r


...

ERROR:root:on_close() called due to IdleTerminate: 20


{'message_id': 2115, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487913, 'text': '/start', 'entities': [{'offset': 0, 'length': 6, 'type': 'bot_command'}]}
/start


ERROR:root:on_close() called due to IdleTerminate: 20


{'message_id': 2117, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487964, 'text': 'problem to sincronize time with server'}
problem to sincronize time with server
{'message_id': 2120, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 1641487976, 'text': 'y'}
y
{'message_id': 2123, 'from': {'id': 102199992, 'is_bot': False, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'language_code': 'it'}, 'chat': {'id': 102199992, 'first_name': 'Marco', 'last_name': 'Lilliu', 'username': 'Marco_Lilliu', 'type': 'private'}, 'date': 16

ERROR:root:on_close() called due to IdleTerminate: 20
